<h3>Cleaning and Preprocessing</h3>

In [1]:
#import
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import pandas as pd
import regex as re
import string

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
# Define the column names
column_names = ['tweet_id', 'entity', 'sentiment', 'content']

dataset_path = 'twitter_training.csv' 
#eval_data_path = 'twitter_evaluation.csv'
tweets_df = pd.read_csv(dataset_path, header=None, names=column_names)
#eval_df = pd.read_csv(dataset_path, header=None, names=column_names)

tweets_df.head()

,tweet_id,entity,sentiment,content
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...


In [3]:
print("Before: ")

print("null values: ", tweets_df.isnull().sum())
print("duplicates: ", tweets_df.duplicated().sum())

tweets_df.dropna(inplace=True)
tweets_df.drop_duplicates(inplace=True)
tweets_df.drop('tweet_id', axis=1, inplace=True)

#for evaluation data
#eval_df.dropna(inplace=True)
#eval_df.drop_duplicates(inplace=True)
#eval_df.drop('tweet_id', axis=1, inplace=True)

Before: 
null values:  tweet_id       0
entity         0
sentiment      0
content      686
dtype: int64
duplicates:  2700


In [4]:
# Remove Non-String
def filter_non_string(df, column):
    df = df.dropna(subset=[column])
    df[column] = df[column].astype(str)
    return df

# Convert In LowerCase
def normalize_text(text):
    """Convert text to lowercase to ensure consistency across the corpus."""
    return text.lower()

# Remove HTML Tags
def remove_html_tags(text):
    """Remove HTML tags from text."""
    return re.sub(r'<.*?>', '', text)

# Remove URL Or HyperLink
def remove_urls(text):
    """Remove URLs or hyperlinks from the text."""
    return re.sub(r'http\S+|www\S+', '', text)

# Remove Punctuation
def remove_punctuation(text):
    """Remove punctuation marks from the text."""
    return text.translate(str.maketrans('', '', string.punctuation))

# Split Text In Token
def tokenize_text(text):
    """Split the text into individual words or tokens."""
    return word_tokenize(text)

# Eliminate Stopwords
def remove_stopwords(tokens):
    """Eliminate common stopwords from the tokenized text."""
    stop_words = set(stopwords.words('english'))
    return [word for word in tokens if word not in stop_words]

# Remove Emojis
def remove_emojis(text):
    """Remove emojis from the text."""
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

# Stem Tokens
def stem_tokens(tokens):
    """Apply stemming to the tokenized text."""
    stemmer = PorterStemmer()
    return [stemmer.stem(word) for word in tokens]

# Lemmatize Tokens
def lemmatize_tokens(tokens):
    """Apply lemmatization to the tokenized text."""
    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(word) for word in tokens]

def preprocess_text(df):
    df = filter_non_string(df, 'content')
    df['cleaned_text'] = df['content'].apply(normalize_text)
    df['cleaned_text'] = df['cleaned_text'].apply(remove_html_tags)
    df['cleaned_text'] = df['cleaned_text'].apply(remove_urls)
    df['cleaned_text'] = df['cleaned_text'].apply(remove_punctuation)
    df['cleaned_text'] = df['cleaned_text'].apply(remove_emojis)
    df['cleaned_text'] = df['cleaned_text'].apply(tokenize_text)
    df['cleaned_text'] = df['cleaned_text'].apply(remove_stopwords)
    #df['cleaned_text'] = df['cleaned_text'].apply(stem_tokens)
    df['cleaned_text'] = df['cleaned_text'].apply(lemmatize_tokens)
    # Optionally, join the tokens back into a string if needed
    df['cleaned_text'] = df['cleaned_text'].apply(lambda tokens: ' '.join(tokens))
    return df

# Usage:
data_processed = preprocess_text(tweets_df)
#eval_processed = preprocess_text(eval_df)
print(data_processed.head())

        entity sentiment                                            content  \
0  Borderlands  Positive  im getting on borderlands and i will murder yo...   
1  Borderlands  Positive  I am coming to the borders and I will kill you...   
2  Borderlands  Positive  im getting on borderlands and i will kill you ...   
3  Borderlands  Positive  im coming on borderlands and i will murder you...   
4  Borderlands  Positive  im getting on borderlands 2 and i will murder ...   

                     cleaned_text  
0    im getting borderland murder  
1              coming border kill  
2      im getting borderland kill  
3     im coming borderland murder  
4  im getting borderland 2 murder  


In [5]:
data_processed

,entity,sentiment,content,cleaned_text
0,Borderlands,Positive,im getting on borderlands and i will murder yo...,im getting borderland murder
1,Borderlands,Positive,I am coming to the borders and I will kill you...,coming border kill
2,Borderlands,Positive,im getting on borderlands and i will kill you ...,im getting borderland kill
3,Borderlands,Positive,im coming on borderlands and i will murder you...,im coming borderland murder
4,Borderlands,Positive,im getting on borderlands 2 and i will murder ...,im getting borderland 2 murder
...,...,...,...,...
74677,Nvidia,Positive,Just realized that the Windows partition of my...,realized window partition mac like 6 year behi...
74678,Nvidia,Positive,Just realized that my Mac window partition is ...,realized mac window partition 6 year behind nv...
74679,Nvidia,Positive,Just realized the windows partition of my Mac ...,realized window partition mac 6 year behind nv...
74680,Nvidia,Positive,Just realized between the windows partition of...,realized window partition mac like 6 year behi...


In [6]:
from sklearn.preprocessing import LabelEncoder

# Initialize the LabelEncoder
le = LabelEncoder()

# Encode the 'Entity' feature
data_processed['entity'] = le.fit_transform(data_processed['entity'])
#eval_processed['entity'] = le.fit_transform(eval_processed['entity'])

# Encode the 'Sentiment' target variable
data_processed['sentiment'] = le.fit_transform(data_processed['sentiment'])
#eval_processed['sentiment'] = le.fit_transform(eval_processed['sentiment'])

In [7]:
data_processed.head()

,entity,sentiment,content,cleaned_text
0,4,3,im getting on borderlands and i will murder yo...,im getting borderland murder
1,4,3,I am coming to the borders and I will kill you...,coming border kill
2,4,3,im getting on borderlands and i will kill you ...,im getting borderland kill
3,4,3,im coming on borderlands and i will murder you...,im coming borderland murder
4,4,3,im getting on borderlands 2 and i will murder ...,im getting borderland 2 murder


In [8]:
#saving the preprocessed data
data_processed.to_csv('preprocessed_tweets.csv', index=False)
#eval_processed.to_csv('processed_evaluation_tweets.csv', index=False)

<h3>Encoding</h3>

In [9]:
#importing the libraries for encoding
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec
import numpy as np

In [10]:
#importing the preprocessed data
preprocessed_df = pd.read_csv('preprocessed_tweets.csv')
#eval_processed = pd.read_csv('processed_evaluation_tweets.csv')

preprocessed_df.head()

,entity,sentiment,content,cleaned_text
0,4,3,im getting on borderlands and i will murder yo...,im getting borderland murder
1,4,3,I am coming to the borders and I will kill you...,coming border kill
2,4,3,im getting on borderlands and i will kill you ...,im getting borderland kill
3,4,3,im coming on borderlands and i will murder you...,im coming borderland murder
4,4,3,im getting on borderlands 2 and i will murder ...,im getting borderland 2 murder


In [11]:
#Bag of words
def encode_bow(corpus):
    vectorizer = CountVectorizer()
    bow_vectors = vectorizer.fit_transform(corpus)
    return bow_vectors, vectorizer
#TF-IDF
def encode_tfidf(corpus):
    vectorizer = TfidfVectorizer()
    tfidf_vectors = vectorizer.fit_transform(corpus)
    return tfidf_vectors, vectorizer
# Word2Vec (CBOW and Skip-Gram)
def train_word2vec(corpus, vector_size=100, window=5, min_count=1, sg=0):
    tokenized_corpus = [doc.split() for doc in corpus]
    model = Word2Vec(sentences=tokenized_corpus, vector_size=vector_size, window=window, min_count=min_count, sg=sg)
    return model

def encode_word2vec(model, tokenized_corpus):
    vectors = []
    for tokens in tokenized_corpus:
        vector = sum([model.wv[token] for token in tokens if token in model.wv], start=np.zeros(model.vector_size))
        vectors.append(vector)
    return np.array(vectors)

preprocessed_df['cleaned_text'].fillna('', inplace=True)

# Encode using Bag of Words
bow_vectors, bow_vectorizer = encode_bow(data_processed['cleaned_text'])

# Encode using TF-IDF
tfidf_vectors, tfidf_vectorizer = encode_tfidf(data_processed['cleaned_text'])

# Train Word2Vec models (CBOW and Skip-Gram)
cbow_model = train_word2vec(data_processed['cleaned_text'], sg=0)
skipgram_model = train_word2vec(data_processed['cleaned_text'], sg=1)

# Tokenized corpus for Word2Vec encoding
tokenized_corpus = [doc.split() for doc in data_processed['cleaned_text']]

# Encode using Word2Vec (CBOW)
cbow_vectors = encode_word2vec(cbow_model, tokenized_corpus)

# Encode using Word2Vec (Skip-Gram)
skipgram_vectors = encode_word2vec(skipgram_model, tokenized_corpus)

# Display the shapes of the encoded vectors to confirm
print("BoW Vectors Shape:", bow_vectors.shape)
print("TF-IDF Vectors Shape:", tfidf_vectors.shape)
print("Word2Vec CBOW Vectors Shape:", cbow_vectors.shape)
print("Word2Vec Skip-Gram Vectors Shape:", skipgram_vectors.shape)

BoW Vectors Shape: (71656, 36479)
TF-IDF Vectors Shape: (71656, 36479)
Word2Vec CBOW Vectors Shape: (71656, 100)
Word2Vec Skip-Gram Vectors Shape: (71656, 100)


<h3>Model Training</h3>

In [12]:
#importing libraries
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score, classification_report
import numpy as np

In [22]:
# Combine tokenized answers with other features
features = np.hstack([cbow_vectors, preprocessed_df[['entity']].values])

target_variable = preprocessed_df['sentiment']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target_variable, test_size=0.2, random_state=42)
X_train

In [23]:
X_train

array([[13.88178167, -4.41066475, -2.67998901, ..., -7.00514266,
        -0.61301113,  2.54021149],
       [ 1.47821298,  0.0922516 ,  0.50117174, ...,  1.76177269,
        -0.73954082,  1.76144001],
       [ 0.45390699, -0.16059101,  1.24430284, ..., -0.25214958,
        -0.4659901 ,  0.8960343 ],
       ...,
       [ 1.54340419,  0.55920236,  1.07455769, ...,  2.91723907,
        -2.51497099,  1.47535869],
       [ 4.97575417, -1.11072598, -0.26299217, ..., -1.42512081,
         1.56929128,  0.04258987],
       [ 3.45107031, -0.64883866, -0.033836  , ..., -0.69993597,
         0.75943356,  1.23980311]])

In [24]:
# Function to train SVM
def train_svm(X_train, y_train, kernel='rbf', C=1.0):
    svm = SVC(kernel=kernel, C=C)
    svm.fit(X_train, y_train)
    return svm

# Function to train Naive Bayes
def train_naive_bayes(X_train, y_train):
    nb = GaussianNB()
    nb.fit(X_train, y_train)
    return nb

# Function to train Logistic Regression
def train_logistic_regression(X_train, y_train):
    lr = LogisticRegression(max_iter=1000)
    lr.fit(X_train, y_train)
    return lr

# Function to train AdaBoost
def train_adaboost(X_train, y_train, n_estimators=50):
    ab = AdaBoostClassifier(n_estimators=n_estimators)
    ab.fit(X_train, y_train)
    return ab


In [21]:
# Train models
nb_model = train_naive_bayes(X_train, y_train)
lr_model = train_logistic_regression(X_train, y_train)
ab_model = train_adaboost(X_train, y_train)
svm_model = train_svm(X_train, y_train)

# Make predictions on the testing set
svm_pred = svm_model.predict(X_test)
nb_pred = nb_model.predict(X_test)
lr_pred = lr_model.predict(X_test)
ab_pred = ab_model.predict(X_test)

# Calculate the accuracy of the models
svm_accuracy = accuracy_score(y_test, svm_pred)
nb_accuracy = accuracy_score(y_test, nb_pred)
lr_accuracy = accuracy_score(y_test, lr_pred)
ab_accuracy = accuracy_score(y_test, ab_pred)

# Generate the classification reports
svm_report = classification_report(y_test, svm_pred)
nb_report = classification_report(y_test, nb_pred)
lr_report = classification_report(y_test, lr_pred)
ab_report = classification_report(y_test, ab_pred)

print("SVM Accuracy:", svm_accuracy)
print("SVM Classification Report:\n", svm_report)
print('*' * 20)

print("Naive Bayes Accuracy:", nb_accuracy)
print("Naive Bayes Classification Report:\n", nb_report)
print('*' * 20)

print("Logistic Regression Accuracy:", lr_accuracy)
print("Logistic Regression Classification Report:\n", lr_report)
print('*' * 20)

print("AdaBoost Accuracy:", ab_accuracy)
print("AdaBoost Classification Report:\n", ab_report)
print('*' * 20)

C:\Users\HP\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


SVM Accuracy: 0.551911805749372
SVM Classification Report:
               precision    recall  f1-score   support

           0       0.57      0.17      0.26      2529
           1       0.59      0.72      0.65      4383
           2       0.58      0.46      0.52      3543
           3       0.50      0.69      0.58      3877

    accuracy                           0.55     14332
   macro avg       0.56      0.51      0.50     14332
weighted avg       0.56      0.55      0.53     14332

********************
Naive Bayes Accuracy: 0.4330867987719788
Naive Bayes Classification Report:
               precision    recall  f1-score   support

           0       0.49      0.06      0.10      2529
           1       0.53      0.47      0.50      4383
           2       0.50      0.30      0.37      3543
           3       0.37      0.76      0.50      3877

    accuracy                           0.43     14332
   macro avg       0.47      0.40      0.37     14332
weighted avg       0.47    

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline

# Define a function to train and evaluate a pipeline
def train_and_evaluate(X, y, model):
    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Create a pipeline with TF-IDF Vectorizer and the specified model
    pipeline = Pipeline([
        ('tfidf', TfidfVectorizer()),
        ('clf', model)
    ])

    # Fit the model on the training data
    pipeline.fit(X_train, y_train)

    # Make predictions on the testing set
    y_pred = pipeline.predict(X_test)

    # Calculate the accuracy of the model
    accuracy = accuracy_score(y_test, y_pred)

    # Generate the classification report
    report = classification_report(y_test, y_pred)

    print(f"Model: {model.__class__.__name__}")
    print(f"Accuracy: {accuracy}")
    print(f"Classification Report:\n{report}")
    print('\n')

# Specify the models
models = [
    LogisticRegression(),
    MultinomialNB(),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    SVC()
]

# Train and evaluate each model
for model in models:
    train_and_evaluate(X, y, model)

    

C:\Users\HP\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Model: LogisticRegression
Accuracy: 0.7795841473625453
Classification Report:
              precision    recall  f1-score   support

           0       0.81      0.67      0.73      2529
           1       0.78      0.86      0.82      4383
           2       0.78      0.75      0.76      3543
           3       0.76      0.79      0.78      3877

    accuracy                           0.78     14332
   macro avg       0.78      0.77      0.77     14332
weighted avg       0.78      0.78      0.78     14332



